In [54]:
import pandas as pd
import numpy as np
import time
import ast

In [55]:
start = time.time()
df = pd.read_csv('./amazon_uk_shoes_products_dataset_2021_12.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6823 entries, 0 to 6822
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   url                6823 non-null   object 
 1   product_name       6823 non-null   object 
 2   reviewer_name      6823 non-null   object 
 3   review_title       6822 non-null   object 
 4   review_text        6814 non-null   object 
 5   review_rating      6823 non-null   float64
 6   verified_purchase  6823 non-null   bool   
 7   review_date        6823 non-null   object 
 8   helpful_count      1953 non-null   object 
 9   uniq_id            6823 non-null   object 
 10  scraped_at         6823 non-null   object 
dtypes: bool(1), float64(1), object(9)
memory usage: 539.8+ KB


In [56]:
df.isnull()
df.isnull().sum()#결측치 확인

url                     0
product_name            0
reviewer_name           0
review_title            1
review_text             9
review_rating           0
verified_purchase       0
review_date             0
helpful_count        4870
uniq_id                 0
scraped_at              0
dtype: int64

In [ ]:
df.drop(["helpful_count"], axis = 1, inplace=True) #결측치가 너무 많은 열이라서 제거
df = df.dropna(subset=['review_title', 'review_text'])#review_title, review_text의 결측치 제거
df.isnull().sum()#모든 결측치가 사라짐을 확인

In [ ]:
df.drop_duplicates() # 중복 데이터 제거
df

In [85]:
from nltk.tokenize import word_tokenize #리뷰글들의 불용어를 확인하기 위해 review_text를 토큰화 하였습니다.
nltk.download('punkt')
tokens = sent_tokenize("review_text")
print("tokens")

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


tokens


In [86]:
%pip install nltk
import nltk as nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer # nltk의 VADER 감정 분석기를 가져옮
nltk.download('vader_lexicon')
nltk.download('stopwords')
start = time.time()
cache = set(stopwords.words("english"))
def remove_stopwords(review):
    text = " ".join([word for word in review.split() if word not in cache])
    return text
df.review_text = df.review_text.apply(remove_stopwords)

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [76]:
vader_sentiment = SentimentIntensityAnalyzer() # VADER 감정 분석기를 초기화
def calc_sentiment(review):    
    if review == "No Negative" or review == "No Positive": # 리뷰가 "No Negative" 또는 "No Positive"인 경우
        return 0
    return vader_sentiment.polarity_scores(review)["compound"]  # 리뷰가 텍스트인 경우 VADER 감정 분석기를 사용하여 감정 점수(컴파운드 스코어)를 계산하고 반환

In [87]:
print("Calculating sentiment columns for review_text")
start = time.time()
df["review_text_Sentiment"] = df.review_text.apply(calc_sentiment)
end = time.time()
print("Calculating sentiment took " + str(round(end - start, 2)) + " seconds")

Calculating sentiment columns for review_text
Calculating sentiment took 1.89 seconds


In [88]:
df = df.sort_values(by=["review_text_Sentiment"], ascending=True)
print(df[["review_text_Sentiment"]])

      review_text_Sentiment
2103                -0.9955
5328                -0.9930
5773                -0.9917
5772                -0.9917
2080                -0.9914
...                     ...
5427                 0.9949
5197                 0.9956
1335                 0.9963
3298                 0.9964
5041                 0.9971

[6813 rows x 1 columns]
